## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [2]:
# If you didn't download it yet, please get the relevant file now!
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

## Explanation
### Dataset
Several channels measuring time, internal cloc, address (FPGA(01)+TOC_CHANNEL), measurements: tdc -> every tick is a (25/30)ns ad 30 BX_counter bumps up. When BX_counter goes to somehting it bumps up Orbit_CNT...

See theory lesson for details

---

In [1]:
import pandas as pd
import random



In [3]:
# 1
raw = pd.read_csv('./data/data_000637.txt')

df1 = raw.sample(random.randrange(10001, raw.shape[0]), axis=0)
display(df1)

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
1301909,1,0,100,3869211114,419,23
35283,1,1,46,3869200518,1850,2
857561,1,0,59,3869208059,611,5
1017730,1,0,78,3869209144,2760,23
1157508,1,0,36,3869210108,2548,3
...,...,...,...,...,...,...
200313,1,0,122,3869202160,1012,6
19337,1,1,139,3869200360,589,0
739926,1,0,60,3869207267,3353,13
1087429,1,0,63,3869209630,1033,27


In [5]:
# 2 
print('BX_COUNTER MAX', raw['BX_COUNTER'].max(), '\n')
print(raw.min())


BX_COUNTER MAX 3563 

HEAD                    1
FPGA                    0
TDC_CHANNEL             1
ORBIT_CNT      3869200167
BX_COUNTER              0
TDC_MEAS                0
dtype: int64


In [4]:
# 3
display(raw)

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14
1310716,1,1,4,3869211171,763,11
1310717,1,0,64,3869211171,764,0
1310718,1,0,139,3869211171,769,0


In [6]:
ini = raw.head(1)
fin = raw.tail(1)

fini = ((fin['ORBIT_CNT']*3564/30 + fin['BX_COUNTER'] + fin['TDC_MEAS']/30)*25).iloc[0]
inif = ((ini['ORBIT_CNT']*3564/30 + ini['BX_COUNTER'] + ini['TDC_MEAS']/30)*25).iloc[0]

print(fini - inif)


32641573.333984375


In [16]:
df1['ABS_TIME']=(df1['ORBIT_CNT']*3564/30 + df1['BX_COUNTER'] + df1['TDC_MEAS']/30)*25

In [17]:
display(df1)

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABS_TIME
1301909,1,0,100,3869211114,419,23,1.149156e+13
35283,1,1,46,3869200518,1850,2,1.149153e+13
857561,1,0,59,3869208059,611,5,1.149155e+13
1017730,1,0,78,3869209144,2760,23,1.149155e+13
1157508,1,0,36,3869210108,2548,3,1.149155e+13
...,...,...,...,...,...,...,...
200313,1,0,122,3869202160,1012,6,1.149153e+13
19337,1,1,139,3869200360,589,0,1.149153e+13
739926,1,0,60,3869207267,3353,13,1.149155e+13
1087429,1,0,63,3869209630,1033,27,1.149155e+13


In [71]:
df1.assign(HEAD=lambda x: random.sample(0,1))

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
653981,1,0,48,3869206660,3345,4
624561,1,0,56,3869206481,2117,26
507534,1,1,13,3869205299,687,16
796144,1,1,40,3869207651,1509,18
1180057,1,0,67,3869210287,3394,10
...,...,...,...,...,...,...
363688,1,0,64,3869203817,698,26
193544,1,1,31,3869202093,2029,12
179159,1,0,36,3869201949,3258,15
851701,1,0,28,3869208017,2764,0


In [70]:
display(df1)

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
653981,1,0,48,3869206660,3345,4
624561,1,0,56,3869206481,2117,26
507534,1,1,13,3869205299,687,16
796144,1,1,40,3869207651,1509,18
1180057,1,0,67,3869210287,3394,10
...,...,...,...,...,...,...
363688,1,0,64,3869203817,698,26
193544,1,1,31,3869202093,2029,12
179159,1,0,36,3869201949,3258,15
851701,1,0,28,3869208017,2764,0
